In [ ]:
from collections import Counter, defaultdict
import itertools
import math
import re

In [ ]:
filename = "day20.example.input"
filename = "day20.input"

with open(filename) as file:
    tile_parts = file.read().strip().split("\n\n")

In [ ]:
def rotate_data(data, times=1):
    """Rotate data 90 degrees clockwise."""
    for _ in range(times):
        data = ["".join(line) for line in zip(*data[::-1])]
    return data

def flip_data(data, axis=0):
    """Flip data along axis."""
    if axis == 0:
        return data[::-1]
    if axis == 1:
        return [row[::-1] for row in data]

In [ ]:
class Tile:
    def __init__(self, input_string):
        lines = input_string.strip().split("\n")
        self.id = int(re.search("(\d+)", lines[0]).group(1))
        self.data = lines[1:]
    
    @property
    def normal_edges(self):
        return (
            self.data[0], self.data[-1],
            "".join(line[0] for line in self.data),
            "".join(line[-1] for line in self.data),
        )
        
    @property
    def edges(self):
        return self.normal_edges + tuple(edge[::-1] for edge in self.normal_edges)
    
    @property
    def image(self):
        return [row[1:-1] for row in self.data[1:-1]]
    
    def rotate(self, times=1):
        self.data = rotate_data(self.data, times=times)
        
    def flip(self, axis=0):
        self.data = flip_data(self.data, axis)

In [ ]:
tiles = dict()
for part in tile_parts:
    tile = Tile(part)
    tiles[tile.id] = tile

In [ ]:
edge_counts = Counter(
    itertools.chain.from_iterable(tile.edges for tile in tiles.values())
)

In [ ]:
# All edges within a tile are unique
for tile in tiles.values():
    assert len(tile.edges) == 8

# Every edge occurs either 1 or 2 times
assert set(edge_counts.values()) == {1, 2}

# Number of pairs equals number of joints
square_dim = int(math.sqrt(len(tiles)))
assert Counter(edge_counts.values())[2] == 2*2*square_dim*(square_dim - 1)

# Part 1

In [ ]:
# A corner tile is a tile where 4 edges are unique (not shared with another tile)
corner_tiles = []
for tile in tiles.values():
    if Counter(edge_counts[edge] for edge in tile.edges)[1] == 4:
        corner_tiles.append(tile.id)

math.prod(corner_tiles)

# Part 2

In [ ]:
def rotate_and_flip(tile):
    for _ in range(2):
        yield
        for _ in range(3):
            tile.rotate()
            yield
        tile.flip()

def edge_neighbours(tile):
    return tuple(edge_counts[edge] for edge in tile.normal_edges)

def align_ul_corner(tile):
    for _ in rotate_and_flip(tile):
        if edge_neighbours(tile) == (1, 2, 1, 2):
            break
    else:
        raise ValueError("Unable to align corner")

In [ ]:
# Dictionary that can find which tile has which edge
has_edge = defaultdict(set)
for tile in tiles.values():
    for edge in tile.edges:
        has_edge[edge].add(tile.id)

In [ ]:
corner_tiles

In [ ]:
grid = [[None]*square_dim for _ in range(square_dim)]

# Select a corner tile for the upper left corner
tile = tiles[corner_tiles[1]]
align_ul_corner(tile)
grid[0][0] = tile.id

for row in range(square_dim):
    for col in range(square_dim):
        
        if not grid[row][col] is None:
            # Just skip past the UL corner
            continue
        
        if row == 0:
            # Left edge must match previous, upper edge has no neighbours
            left_tile = tiles[grid[row][col - 1]]
            left_edge = left_tile.normal_edges[3]  # Right edge of left tile
            tile = tiles[(has_edge[left_edge] - {left_tile.id}).pop()]
            for _ in rotate_and_flip(tile):
                if (tile.normal_edges[2] == left_edge) and (edge_neighbours(tile)[0] == 1):
                    break            
        else:
            
            if col == 0:
                # Left edge has no neighbours, upper edge must match above
                tile_above = tiles[grid[row - 1][col]]
                upper_edge = tile_above.normal_edges[1]  # lower edge of tile above
                tile = tiles[(has_edge[upper_edge] - {tile_above.id}).pop()]
                for _ in rotate_and_flip(tile):
                    if (tile.normal_edges[0] == upper_edge) and (edge_neighbours(tile)[2] == 1):
                        break
            else:
                # Left edge must match previous, upper edge must match above
                left_tile = tiles[grid[row][col - 1]]
                left_edge = left_tile.normal_edges[3]  # Right edge of left tile
                tile_above = tiles[grid[row - 1][col]]
                upper_edge = tile_above.normal_edges[1]  # lower edge of tile above
                tile = tiles[(has_edge[left_edge] - {left_tile.id}).pop()]
                for _ in rotate_and_flip(tile):
                    if (tile.normal_edges[0] == upper_edge) and (tile.normal_edges[2] == left_edge):
                        break
        
        grid[row][col] = tile.id

grid

In [ ]:
image = []
for row in grid:
    image.extend(["".join(line) for line in zip(*[tiles[col].image for col in row])])

print("\n".join(image))

In [ ]:
MONSTER = [
    "                  # ",
    "#    ##    ##    ###",
    " #  #  #  #  #  #   "
]

In [ ]:
def find_monsters(image):
    monsters = 0
    for startrow in range(0, len(image) - len(MONSTER) + 1):
        for startcol in range(0, len(image[0]) - len(MONSTER[0]) + 1):
            window = [row[startcol:startcol + len(MONSTER[0])] for row in image[startrow:startrow + len(MONSTER)]]
            for img, mon in zip("".join(window), "".join(MONSTER)):
                if (mon == "#") and (img != "#"):
                    break
            else:
                monsters += 1
    return monsters

In [ ]:
def rotate_and_flip_image(image):
    for _ in range(2):
        yield image
        for _ in range(3):
            image = rotate_data(image)
            yield image
        image = flip_data(image)

In [ ]:
for image in rotate_and_flip_image(image):
    if find_monsters(image) > 0:
        break

In [ ]:
"".join(image).count("#") - find_monsters(image) * "".join(MONSTER).count("#")